In [24]:
# %load A8_VAE.py
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import torch.backends.cudnn as cudnn
import os
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"] = str(4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True
torch.manual_seed(2)


train_loader = torch.utils.data.DataLoader(datasets.MNIST('../data', train=True, download=True,
                                            transform=transforms.ToTensor()),batch_size=128, shuffle=True, num_workers=0)

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc2_1 = nn.Linear(400, 20)
        self.fc2_2 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc2_1(h1), self.fc2_2(h1)

    def reparameterize(self, mu, logvar):
        ##############################
        ## implement the reparameterize function
        ##############################
        std = torch.sqrt(torch.exp(logvar))
        #m = torch.distributions.normal.Normal(torch.zeros(1, 20), torch.ones(1, 20))
        #eps = m.sample()
        #eps = torch.normal(1,0,size=(1,20))
        eps = torch.randn(20)
        return mu + std * eps

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

def criterion(recon_x, x, mu, logvar):
    #BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduce=True, size_average=False)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0)

num_epochs = 30
batch_size = 128

def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = criterion(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('===== Epoch: {} Average loss: {:.4f} ======'.format(
          epoch, train_loss / len(train_loader.dataset)))


if __name__ == "__main__":
    for epoch in range(num_epochs):
        train(epoch)
        with torch.no_grad():
            sample = torch.randn(int(batch_size/2), 20).to(device)
            sample = model.decode(sample)
            save_image(sample.view(int(batch_size/2), 1, 28, 28),
                       'results/sampled_output_' + str(epoch) + '.png')    


Train Epoch: 0 [0/60000 (0%)]	Loss: 549.329163
Train Epoch: 0 [12800/60000 (21%)]	Loss: 208.057144
Train Epoch: 0 [25600/60000 (43%)]	Loss: 178.664627
Train Epoch: 0 [38400/60000 (64%)]	Loss: 162.106674
Train Epoch: 0 [51200/60000 (85%)]	Loss: 147.114807
===== Epoch: 0 Average loss: 188.2827 ======
Train Epoch: 1 [0/60000 (0%)]	Loss: 142.343903
Train Epoch: 1 [12800/60000 (21%)]	Loss: 151.265350
Train Epoch: 1 [25600/60000 (43%)]	Loss: 149.966141
Train Epoch: 1 [38400/60000 (64%)]	Loss: 139.544922
Train Epoch: 1 [51200/60000 (85%)]	Loss: 143.896851
===== Epoch: 1 Average loss: 143.1987 ======
Train Epoch: 2 [0/60000 (0%)]	Loss: 138.414032
Train Epoch: 2 [12800/60000 (21%)]	Loss: 129.915863
Train Epoch: 2 [25600/60000 (43%)]	Loss: 136.132095
Train Epoch: 2 [38400/60000 (64%)]	Loss: 130.150269
Train Epoch: 2 [51200/60000 (85%)]	Loss: 128.486725
===== Epoch: 2 Average loss: 131.6460 ======
Train Epoch: 3 [0/60000 (0%)]	Loss: 120.067657
Train Epoch: 3 [12800/60000 (21%)]	Loss: 126.343567
T

Train Epoch: 27 [12800/60000 (21%)]	Loss: 109.016304
Train Epoch: 27 [25600/60000 (43%)]	Loss: 109.043495
Train Epoch: 27 [38400/60000 (64%)]	Loss: 114.405289
Train Epoch: 27 [51200/60000 (85%)]	Loss: 103.889381
===== Epoch: 27 Average loss: 108.2111 ======
Train Epoch: 28 [0/60000 (0%)]	Loss: 114.391190
Train Epoch: 28 [12800/60000 (21%)]	Loss: 105.927559
Train Epoch: 28 [25600/60000 (43%)]	Loss: 112.804100
Train Epoch: 28 [38400/60000 (64%)]	Loss: 103.833298
Train Epoch: 28 [51200/60000 (85%)]	Loss: 114.626579
===== Epoch: 28 Average loss: 107.6759 ======
Train Epoch: 29 [0/60000 (0%)]	Loss: 101.503197
Train Epoch: 29 [12800/60000 (21%)]	Loss: 101.213989
Train Epoch: 29 [25600/60000 (43%)]	Loss: 104.459053
Train Epoch: 29 [38400/60000 (64%)]	Loss: 103.321243
Train Epoch: 29 [51200/60000 (85%)]	Loss: 105.672241
===== Epoch: 29 Average loss: 107.6429 ======
